In [1]:
import os
import pandas as pd
import numpy as np
import cv2

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import pickle
import csv

from sklearn.metrics import confusion_matrix, classification_report


# from keras.utils import to_categorical
# encoded_labels = to_categorical(np.asarray(labels))


In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  # or even "-1"

import tensorflow as tf
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
K.set_learning_phase(1)


from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


# import tensorflow as tf
# print(tf.test.is_built_with_cuda())

# import tensorflow as tf
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# tf.test.is_gpu_available( cuda_only=False, min_cuda_compute_capability=None)

Using TensorFlow backend.
W0921 20:59:39.786558 13588 deprecation_wrapper.py:119] From c:\users\ayush\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:153: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6055533642135848900
]


In [4]:
chexnet_targets = ['No Finding',
       'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices']

chexpert_targets = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']

In [5]:
u_one_features = ['Atelectasis', 'Edema']
u_zero_features = ['Cardiomegaly', 'Consolidation', 'Pleural Effusion']

In [6]:
pd.set_option('display.max_columns', None)  # or 1000
# pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

In [7]:
full_train_df = pd.read_csv("E:/Minor/CheXpert-v1.0-small/train.csv")
full_valid_df = pd.read_csv("E:/Minor/CheXpert-v1.0-small/valid.csv")

# full_train_df.head()
full_valid_df.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/valid/patient64541/study1/view1_frontal.jpg,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CheXpert-v1.0-small/valid/patient64542/study1/view1_frontal.jpg,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,CheXpert-v1.0-small/valid/patient64542/study1/view2_lateral.jpg,Male,70,Lateral,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,CheXpert-v1.0-small/valid/patient64543/study1/view1_frontal.jpg,Male,85,Frontal,AP,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CheXpert-v1.0-small/valid/patient64544/study1/view1_frontal.jpg,Female,42,Frontal,AP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
def feature_string(row):
    feature_list = []
    for feature in u_one_features:
        if row[feature] in [-1,1]:
            feature_list.append(feature)
            
    for feature in u_zero_features:
        if row[feature] == 1:
            feature_list.append(feature)
            
    return ';'.join(feature_list)

In [9]:
def get_full_data():
    
    full_train_df['patient'] = full_train_df.Path.str.split('/',3,True)[2]
    full_train_df  ['study'] = full_train_df.Path.str.split('/',4,True)[3]

    full_valid_df['patient'] = full_valid_df.Path.str.split('/',3,True)[2]
    full_valid_df  ['study'] = full_valid_df.Path.str.split('/',4,True)[3]

    full_df = pd.concat([full_train_df, full_valid_df])
    full_df['feature_string'] = full_df.apply(feature_string,axis = 1).fillna('')

    condition = full_df['feature_string'] != ""
    five_df = full_df[condition]
    
#     at = [row for index, row in five_df.iterrows() if disease in row.feature_string]
#     ed = [row for index, row in five_df.iterrows() if 'Edema' in row.feature_string]
#     car = [row for index, row in five_df.iterrows() if 'Cardiomegaly' in row.feature_string]
#     ple = [row for index, row in five_df.iterrows() if 'Pleural Effusion' in row.feature_string]
#     con = [row for index, row in five_df.iterrows() if 'Consolidation' in row.feature_string]

    return five_df



In [10]:
img_width, img_height = 320,320
# img_width, img_height = 200,200

In [11]:
def get_train_images(data):
    
    labels = []
    images =[]
    static = 'E:/Minor/'
    for index, row in data.iterrows():
        if index > 2500:
            break
#         if row[3] == 'Frontal':
#             images.append(cv2.resize(cv2.imread(os.path.join(static,row.Path),cv2.IMREAD_COLOR), (img_width, img_height)).reshape(img_width, img_height,3))
#             label_array = [0,0,0,0,0]
#             features = row.feature_string.split(';')
#             for i in features:
#                 label_array[chexpert_targets.index(i)] = 1
#             labels.append(label_array)
        
        images.append(cv2.resize(cv2.imread(os.path.join(static,row.Path),cv2.IMREAD_COLOR), (img_width, img_height)).reshape(img_width, img_height,3))
        label_array = [0,0,0,0,0]
        features = row.feature_string.split(';')
        for i in features:
            label_array[chexpert_targets.index(i)] = 1
        labels.append(label_array)
        
    #     labels.append(row.Atelectasis)
         

    print(len(images))
    labels = np.asarray(labels)
    images = np.asarray(images)
    images = images/255
    
    return images,labels



In [12]:
from sklearn.model_selection import train_test_split

full_data = get_full_data()
full_data.head(30)
# print(full_data.shape)

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,patient,study,feature_string
1,CheXpert-v1.0-small/train/patient00002/study2/view1_frontal.jpg,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN,patient00002,study2,Atelectasis;Edema
4,CheXpert-v1.0-small/train/patient00003/study1/view1_frontal.jpg,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,patient00003,study1,Edema
12,CheXpert-v1.0-small/train/patient00007/study1/view1_frontal.jpg,Male,69,Frontal,AP,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,0.0,1.0,patient00007,study1,Atelectasis;Cardiomegaly
13,CheXpert-v1.0-small/train/patient00007/study2/view1_frontal.jpg,Male,69,Frontal,AP,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,0.0,1.0,patient00007,study2,Atelectasis
14,CheXpert-v1.0-small/train/patient00008/study1/view1_frontal.jpg,Male,81,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,patient00008,study1,Pleural Effusion
15,CheXpert-v1.0-small/train/patient00008/study2/view1_frontal.jpg,Male,81,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,1.0,patient00008,study2,Pleural Effusion
16,CheXpert-v1.0-small/train/patient00009/study1/view1_frontal.jpg,Male,76,Frontal,PA,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,patient00009,study1,Atelectasis;Cardiomegaly
17,CheXpert-v1.0-small/train/patient00009/study1/view2_lateral.jpg,Male,76,Lateral,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,patient00009,study1,Atelectasis;Cardiomegaly
25,CheXpert-v1.0-small/train/patient00011/study4/view1_frontal.jpg,Female,19,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,-1.0,1.0,-1.0,NaN,NaN,1.0,patient00011,study4,Atelectasis
26,CheXpert-v1.0-small/train/patient00011/study2/view1_frontal.jpg,Female,19,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,patient00011,study2,Consolidation;Pleural Effusion


In [13]:
images,labels = get_train_images(full_data)


X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)




print(labels[:10])
# print(encoded_labels[:10])
print(labels.shape)

1340
[[1 0 0 1 0]
 [0 0 0 1 0]
 [1 1 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [1 1 0 0 0]
 [1 0 0 0 0]
 [0 0 1 0 1]
 [0 0 0 1 1]]
(1340, 5)


In [14]:
labels[:10]

array([[1, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 0, 1, 1]])

In [15]:
print(X_test[:5])
print(y_test[:5])

[[[[0.03529412 0.03529412 0.03529412]
   [0.03529412 0.03529412 0.03529412]
   [0.03529412 0.03529412 0.03529412]
   ...
   [0.03921569 0.03921569 0.03921569]
   [0.03921569 0.03921569 0.03921569]
   [0.03921569 0.03921569 0.03921569]]

  [[0.03529412 0.03529412 0.03529412]
   [0.03529412 0.03529412 0.03529412]
   [0.03529412 0.03529412 0.03529412]
   ...
   [0.03921569 0.03921569 0.03921569]
   [0.03921569 0.03921569 0.03921569]
   [0.03921569 0.03921569 0.03921569]]

  [[0.03529412 0.03529412 0.03529412]
   [0.03529412 0.03529412 0.03529412]
   [0.03529412 0.03529412 0.03529412]
   ...
   [0.03921569 0.03921569 0.03921569]
   [0.03921569 0.03921569 0.03921569]
   [0.03921569 0.03921569 0.03921569]]

  ...

  [[0.12156863 0.12156863 0.12156863]
   [0.14117647 0.14117647 0.14117647]
   [0.16470588 0.16470588 0.16470588]
   ...
   [0.03921569 0.03921569 0.03921569]
   [0.03921569 0.03921569 0.03921569]
   [0.03921569 0.03921569 0.03921569]]

  [[0.11372549 0.11372549 0.11372549]
   [0.1

In [16]:
print(images[1])
# images[:1].shape
# np.amax(images[1])

[[[0.00784314 0.00784314 0.00784314]
  [0.         0.         0.        ]
  [0.00392157 0.00392157 0.00392157]
  ...
  [0.01176471 0.01176471 0.01176471]
  [0.01176471 0.01176471 0.01176471]
  [0.01568627 0.01568627 0.01568627]]

 [[0.00392157 0.00392157 0.00392157]
  [0.01960784 0.01960784 0.01960784]
  [0.01176471 0.01176471 0.01176471]
  ...
  [0.01176471 0.01176471 0.01176471]
  [0.00784314 0.00784314 0.00784314]
  [0.01176471 0.01176471 0.01176471]]

 [[0.01176471 0.01176471 0.01176471]
  [0.01176471 0.01176471 0.01176471]
  [0.02352941 0.02352941 0.02352941]
  ...
  [0.01568627 0.01568627 0.01568627]
  [0.00392157 0.00392157 0.00392157]
  [0.01176471 0.01176471 0.01176471]]

 ...

 [[0.08235294 0.08235294 0.08235294]
  [0.0745098  0.0745098  0.0745098 ]
  [0.08627451 0.08627451 0.08627451]
  ...
  [0.0627451  0.0627451  0.0627451 ]
  [0.07843137 0.07843137 0.07843137]
  [0.08627451 0.08627451 0.08627451]]

 [[0.09019608 0.09019608 0.09019608]
  [0.08235294 0.08235294 0.08235294]


In [17]:
epochs = 5
batch_size = 32
n_classes = 5

def build_model():
    base_model = densenet.DenseNet121(input_shape=(img_width, img_height, 3),
                                     weights='imagenet',
                                     include_top=False,
                                     pooling='avg')
    for layer in base_model.layers:
      layer.trainable = True

    x = base_model.output
    x = Dense(256, kernel_regularizer=regularizers.l1_l2(0.01), activity_regularizer=regularizers.l2(0.01))(x)
    x = Activation('relu')(x)
    x = Dense(128, kernel_regularizer=regularizers.l1_l2(0.01), activity_regularizer=regularizers.l2(0.01))(x)
    x = Activation('relu')(x)
    predictions = Dense(n_classes, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model

model = build_model()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', 'mse'])

early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
callbacks_list = [early_stop, reduce_lr]


W0921 21:00:35.227068 13588 deprecation_wrapper.py:119] From c:\users\ayush\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0921 21:00:35.495055 13588 deprecation_wrapper.py:119] From c:\users\ayush\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0921 21:00:35.627056 13588 deprecation_wrapper.py:119] From c:\users\ayush\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0921 21:00:35.628054 13588 deprecation_wrapper.py:119] From c:\users\ayush\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecate

In [18]:
model_history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    validation_data=(X_test,y_test),
    callbacks=callbacks_list)

Train on 1072 samples, validate on 268 samples
Epoch 1/5
 256/1072 [======>.......................] - ETA: 42:19 - loss: 136.4523 - acc: 0.5547 - mean_squared_error: 0.2442

KeyboardInterrupt: 

In [ ]:
model.save('Densenet_121_ver1.h5')

In [ ]:
from keras.models import load_model
model = load_model('Densenet_121_ver1.h5')

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_curve

probs = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test,probs)

print('AUC: %.3f' % auc)